# 03 - Model Architecture Validation

Validating the ModernBERT-RGAT joint model architecture:

1. Install dependencies and import modules
2. Build the model from config
3. Parameter count breakdown
4. Smoke test forward pass with dummy data
5. RGAT relation importance inspection
6. BIO label alignment verification
7. Full Dataset to DataLoader to Model pipeline test
8. Adjacency tensor visualization

---

## 1. Dependencies & Imports

In [ ]:
# Installing required packages that may not be present
import subprocess, sys

def install_if_missing(package, pip_name=None):
    try:
        __import__(package)
    except ImportError:
        print(f'Installing {pip_name or package}...')
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', '-q', pip_name or package])

install_if_missing('torch', 'torch')
install_if_missing('transformers', 'transformers')
install_if_missing('spacy', 'spacy')
install_if_missing('yaml', 'PyYAML')

# Download spaCy English model if not present
import spacy
try:
    spacy.load('en_core_web_sm')
except OSError:
    print('Downloading spaCy en_core_web_sm model...')
    subprocess.check_call([sys.executable, '-m', 'spacy', 'download', 'en_core_web_sm'])

print('All dependencies ready.')

In [ ]:
import sys, os

# Set project root explicitly
PROJECT_ROOT = os.path.expanduser('~/SOTA-ModernBERT-RGAT-Joint-Aspect-Sentiment-Extraction-for-Food-Tech-Reviews')
assert os.path.isdir(PROJECT_ROOT), f'Project root not found: {PROJECT_ROOT}'

if PROJECT_ROOT not in sys.path:
    sys.path.insert(0, PROJECT_ROOT)
os.chdir(PROJECT_ROOT)
print(f'Project root: {PROJECT_ROOT}')

import torch
import pandas as pd

from src.model import ModernBERT_RGAT, RGATLayer, ATEHead, ASCHead
from src.dataset import ABSAPreprocessor, ABSADataset, create_dataloader
from src.data_pipeline import load_config

print(f'PyTorch: {torch.__version__}')
print(f'CUDA available: {torch.cuda.is_available()}')
if torch.cuda.is_available():
    print(f'GPU: {torch.cuda.get_device_name(0)}')

print('Imports successful.')

## 2. Load Config & Build Model

In [ ]:
config_path = os.path.join(PROJECT_ROOT, 'configs', 'config.yaml')
config = load_config(config_path)

model = ModernBERT_RGAT(
    model_name=config['model']['backbone'],
    hidden_dim=config['model']['hidden_dim'],
    num_sentiment_classes=config['model']['num_sentiment_classes'],
    num_bio_tags=config['model']['num_bio_tags'],
    num_relations=config['model']['rgat']['num_relations'],
    rgat_dropout=config['model']['rgat']['dropout'],
)

print(f'Model built successfully.')
print(f'  Backbone:   {config["model"]["backbone"]}')
print(f'  Max length: {config["model"]["max_len"]}')

## 3. Parameter Count Breakdown

In [ ]:
param_counts = model.get_param_count()

print('Parameter Count Breakdown:')
print(f"{'='*45}")
for component, count in param_counts.items():
    print(f'  {component:15s}: {count:>12,}')
print(f"{'='*45}")

print(f"\nModernBERT backbone: ~{param_counts['bert']/1e6:.0f}M params")
print(f"RGAT + task heads:   ~{(param_counts['total'] - param_counts['bert'])/1e3:.0f}K params")

## 4. Smoke Test: Forward Pass

In [ ]:
max_len = config['model']['max_len']
batch_size = 2
num_relations = config['model']['rgat']['num_relations']

# Dummy inputs to verify the forward pass works end-to-end
dummy_input_ids = torch.randint(0, 1000, (batch_size, max_len))
dummy_attention_mask = torch.ones(batch_size, max_len, dtype=torch.long)
dummy_adj_matrix = torch.zeros(batch_size, num_relations, max_len, max_len)
dummy_aspect_mask = torch.zeros(batch_size, max_len)
dummy_aspect_mask[:, 3:5] = 1  # simulating aspect at token positions 3-4

print('Running forward pass with dummy data...')
model.eval()
with torch.no_grad():
    outputs = model(
        input_ids=dummy_input_ids,
        attention_mask=dummy_attention_mask,
        adj_matrix=dummy_adj_matrix,
        aspect_mask=dummy_aspect_mask,
    )

print(f'\nForward pass successful.')
print(f'  ATE logits shape:       {outputs["ate_logits"].shape}')      
print(f'  Sentiment logits shape: {outputs["sentiment_logits"].shape}')
print(f'  Graph embeddings shape: {outputs["graph_embeddings"].shape}')
print(f'\n  Expected ATE:       [{batch_size}, {max_len}, 3]')
print(f'  Expected Sentiment: [{batch_size}, 4]')

## 5. RGAT Relation Importance Weights (Initial)

In [ ]:
# Inspecting the learnable relation importance weights before training.
# These should all start at the same value and diverge during training.

rel_weights = model.get_relation_importance()

print('RGAT Relation Importance Weights (pre-training):')
print(f"{'='*40}")
for rel, weight in rel_weights.items():
    bar = '#' * int(weight * 30)
    print(f'  {rel:10s}: {weight:.4f}  {bar}')
print(f"{'='*40}")

print('\nAll weights start at sigmoid(1.0) = 0.7311.')
print('After training, amod/neg/advmod should have higher weights.')

## 6. BIO Label Alignment Verification

In [ ]:
# Initializing the preprocessor with the same config the model uses
preprocessor = ABSAPreprocessor(
    model_name=config['model']['backbone'],
    max_len=config['model']['max_len'],
)

# Testing BIO alignment on three cases:
# single-word aspect, multi-word aspect, and implicit aspect
test_cases = [
    {
        'text': 'The spicy ramen was incredibly delicious.',
        'aspect': 'ramen',
        'span_start': 10,
        'span_end': 15,
        'description': 'Single-word aspect'
    },
    {
        'text': 'The delivery time was very fast.',
        'aspect': 'delivery time',
        'span_start': 4,
        'span_end': 17,
        'description': 'Multi-word aspect'
    },
    {
        'text': 'Great food and excellent atmosphere.',
        'aspect': '[ASPECT]',
        'span_start': 0,
        'span_end': 0,
        'description': 'Implicit aspect'
    },
]

label_names = {-100: 'IGN', 0: 'O', 1: 'B-ASP', 2: 'I-ASP'}

for tc in test_cases:
    print(f"\n{'='*60}")
    print(f"Test: {tc['description']}")
    print(f"Text: \"{tc['text']}\"")
    print(f"Aspect: \"{tc['aspect']}\" [{tc['span_start']}:{tc['span_end']}]")
    
    bio_labels, aspect_mask = preprocessor.align_bio_labels(
        tc['text'], tc['aspect'], tc['span_start'], tc['span_end']
    )
    
    # Decoding tokens for display
    encoding = preprocessor.tokenizer(
        tc['text'],
        add_special_tokens=True,
        truncation=True,
        max_length=config['model']['max_len'],
    )
    tokens = preprocessor.tokenizer.convert_ids_to_tokens(encoding.input_ids)
    
    print(f"\n  {'Token':15s} {'BIO Label':10s} {'Aspect Mask':12s}")
    print(f"  {'-'*40}")
    for i, tok in enumerate(tokens):
        lbl = bio_labels[i].item()
        msk = aspect_mask[i].item()
        lbl_name = label_names.get(lbl, str(lbl))
        marker = '  <--' if lbl in [1, 2] else ''
        print(f'  {tok:15s} {lbl_name:10s} {msk:12.0f}{marker}')

print(f"\n{'='*60}")
print('BIO alignment verification complete.')

## 7. Dataset + DataLoader Integration Test

In [ ]:
# Building a small test DataFrame to verify the full pipeline
test_df = pd.DataFrame([
    {'sentence_id': '0001', 'sentence': 'The spicy ramen was incredibly delicious.',
     'aspect': 'ramen', 'polarity': 'positive', 'span_start': 10, 'span_end': 15},
    {'sentence_id': '0002', 'sentence': 'The delivery time was very fast.',
     'aspect': 'delivery time', 'polarity': 'positive', 'span_start': 4, 'span_end': 17},
    {'sentence_id': '0003', 'sentence': 'Bad service and rude staff.',
     'aspect': 'service', 'polarity': 'negative', 'span_start': 4, 'span_end': 11},
    {'sentence_id': '0004', 'sentence': 'Overall a decent place.',
     'aspect': '[ASPECT]', 'polarity': 'neutral', 'span_start': 0, 'span_end': 0},
])

label_map = config['labels']['polarity']

dataloader = create_dataloader(
    df=test_df,
    preprocessor=preprocessor,
    label_map=label_map,
    batch_size=2,
    shuffle=False,
)

print(f'DataLoader created: {len(dataloader)} batches\n')

for batch_idx, batch in enumerate(dataloader):
    print(f'Batch {batch_idx + 1}:')
    for key, val in batch.items():
        print(f'  {key:18s}: shape={str(val.shape):20s}  dtype={val.dtype}')
    
    # Running forward pass through the model
    model.eval()
    with torch.no_grad():
        outputs = model(
            input_ids=batch['input_ids'],
            attention_mask=batch['attention_mask'],
            adj_matrix=batch['adj_matrix'],
            aspect_mask=batch['aspect_mask'],
        )
    print(f'  > ATE logits:       {outputs["ate_logits"].shape}')
    print(f'  > Sentiment logits: {outputs["sentiment_logits"].shape}')
    print()

print('Full pipeline test passed: DataFrame -> Dataset -> DataLoader -> Model')

## 8. Adjacency Tensor Inspection

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

test_text = 'The spicy ramen was incredibly delicious.'
adj = preprocessor.build_adjacency_tensor(test_text)

# Getting tokens for axis labels
enc = preprocessor.tokenizer(test_text, add_special_tokens=True, truncation=True, max_length=config['model']['max_len'])
tokens = preprocessor.tokenizer.convert_ids_to_tokens(enc.input_ids)
n_tokens = len(tokens)

rel_names = ['nsubj', 'amod', 'obj', 'advmod', 'neg', 'compound', 'conj']

# Filtering to only show relations that have edges in this sentence
active_rels = [(i, name) for i, name in enumerate(rel_names) if adj[i, :n_tokens, :n_tokens].sum() > 0]

if active_rels:
    fig, axes = plt.subplots(1, len(active_rels), figsize=(5*len(active_rels), 4))
    if len(active_rels) == 1:
        axes = [axes]
    
    for ax, (rel_idx, rel_name) in zip(axes, active_rels):
        mat = adj[rel_idx, :n_tokens, :n_tokens].numpy()
        sns.heatmap(mat, xticklabels=tokens, yticklabels=tokens, 
                    cmap='Blues', ax=ax, square=True, cbar=False)
        ax.set_title(f'{rel_name} ({rel_idx})', fontsize=11, fontweight='bold')
        ax.tick_params(axis='both', labelsize=8)
    
    plt.suptitle(f'Dependency Relations: \"{test_text}\"', fontsize=13, fontweight='bold', y=1.02)
    plt.tight_layout()
    plt.show()
else:
    print('No active dependency relations found for this sentence.')

print(f'\nAdjacency tensor shape: {adj.shape}')
print(f'Non-zero edges: {(adj > 0).sum().item()}')

---

## Phase 3 Summary

| Component | Status |
|-----------|--------|
| RGATLayer with learnable relation weights | Done |
| ATEHead - BIO tagger | Done |
| ASCHead - CLS + MaxPool | Done |
| ModernBERT_RGAT joint forward | Done |
| Sub-word alignment (-100) | Done |
| ABSADataset + DataLoader | Done |
| Forward pass smoke test | Done |
| Adjacency tensor visualization | Done |

**Next step:** Phase 4 - Training Engine